In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Jackknife:
    def __init__( self, len_data, binsize ):
        self.binsize = binsize
        self.nbins = int(len_data/self.binsize)
        self.N = self.binsize * self.nbins
        self.jack_avg = []
        self.est = 0
        self.var_est = 0

    def set( self, func, list_of_data ):
        for i in range( self.nbins ):
            self.jack_avg.append( func( i, self.binsize, list_of_data ) )

    def do_it( self ):
        for i in range( 0, self.nbins ):
            self.est += self.jack_avg[i]
        self.est /= self.nbins

        for i in range( 0, self.nbins ):
            self.var_est += ( self.jack_avg[i] - self.est )**2
        self.var_est /= self.nbins
        self.var_est *= self.nbins -1

    def mean( self ):
        return self.est

    def var( self ):
        return self.var_est

    def err( self ):
        return np.sqrt(self.var_est)

def simple_mean(i, binsize, np_data):
    resmpld = np.delete(np_data, np.s_[i*binsize:(i+1)*binsize], axis=0)
    return np.mean(resmpld, axis=0)

def format_print(cen, err):
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'

def format_print_w_exact(exact, cen, err):
    if np.abs(err)<1.0e-15:
        return str(cen)+" exact:"+str(exact)
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'+' exact:'+'{ex:.{width}f}'.format(ex=exact, width=i+2)+' ['+'{num:.{width}f}'.format(num=abs(exact-cen)/err, width=2)+' sigma]'

In [3]:
def format_print_w_exact_matrix(exact, cen, err):
    shape=exact.shape
    res=[["",""],["",""]]

    for i in range(shape[0]):
        for j in range(shape[1]):
            res[i][j]=format_print_w_exact(exact[i,j], cen[i,j], err[i,j])
    
    return np.array(res)

In [4]:
def Z2x2(beta,h,kx,ky,kz):
    return 2.0*np.cosh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+8.0*np.cosh(2.0*h)+2.0*(np.exp(4.0*beta*(kx-ky-kz))+np.exp(4.0*beta*(-kx+ky-kz))+np.exp(4.0*beta*(-kx-ky+kz)))

def mag_per_site2x2(beta,h,kx,ky,kz):
    numer = np.sinh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+2.0*np.sinh(2.0*h)
    denom = np.cosh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+4.0*np.cosh(2.0*h)+np.exp(4.0*beta*(kx-ky-kz))+np.exp(4.0*beta*(-kx+ky-kz))+np.exp(4.0*beta*(-kx-ky+kz))
    return -numer/denom

def susc2x2(beta,h,kx,ky,kz):
    numer1 = 4.0*np.cosh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+4.0*np.cosh(2.0*h)
    numer2 = np.sinh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+2.0*np.sinh(2.0*h)
    denom = np.cosh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+4.0*np.cosh(2.0*h)+np.exp(4.0*beta*(kx-ky-kz))+np.exp(4.0*beta*(-kx+ky-kz))+np.exp(4.0*beta*(-kx-ky+kz))
    return numer1/denom - 4.0*(numer2/denom)**2

# delta=1.0e-3
# print("mag")
# numeric=-(np.log(Z2x2(beta, h+delta, kx,ky,kz))-np.log(Z2x2(beta, h-delta, kx,ky,kz)))/(2.0*delta)/4.0
# exact=mag_per_site2x2(beta,h, kx,ky,kz)
# print(format_print_w_exact(exact, numeric, delta**2))
# print("susc")
# numeric=-(mag_per_site2x2(beta,h+delta, kx,ky,kz)-mag_per_site2x2(beta,h-delta, kx,ky,kz))/(2.0*delta)
# exact=susc2x2(beta,h, kx,ky,kz)
# print(format_print_w_exact(exact, numeric, delta**2))

In [5]:
def H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4):
    res = 0.0
    res += kx_*s1*s2
    res += kz_*s1*s3
    res += ky_*s1*s4
    res += ky_*s2*s3
    res += kz_*s2*s4
    res += kx_*s3*s4
    
    res *= -2.0*beta_
    res -= h_*(s1+s2+s3+s4)
    
    return res

In [6]:
def Z2x2_numeric(beta_,h_, kx_,ky_,kz_):
    res=0.0
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    res += np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
    return res

In [7]:
def mag_per_site2x2_numeric(beta_,h_, kx_,ky_,kz_):
    numer=0.0
    denom=0.0
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    numer += (s1+s2+s3+s4)*np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
                    denom += np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
    res = numer/denom/4.0
    return res

In [8]:
def ss_matrix(s1,s2,s3,s4):
    mat=np.array([
        [s1*s4, s1*s3],
        [s1*s1, s1*s2]
    ])
    return mat

In [9]:
def exex_matrix(s1,s2,s3,s4):
    ex=np.array([
        [s4*s3, s3*s4],
        [s1*s2, s2*s1]
    ])
    mat=np.array([
        [ex[0,0]*ex[0,0], ex[0,0]*ex[0,1]],
        [ex[0,0]*ex[1,0], ex[0,0]*ex[1,1]]
    ])
    return mat

In [10]:
def exey_matrix(s1,s2,s3,s4):
    ex=np.array([
        [s4*s3, s3*s4],
        [s1*s2, s2*s1]
    ])
    ey=np.array([
        [s4*s1, s3*s2],
        [s1*s4, s2*s3]
    ])
    mat=np.array([
        [ex[0,0]*ey[0,0], ex[0,0]*ey[0,1]],
        [ex[0,0]*ey[1,0], ex[0,0]*ey[1,1]]
    ])
    return mat

In [11]:
def exez_matrix(s1,s2,s3,s4):
    ex=np.array([
        [s4*s3, s3*s4],
        [s1*s2, s2*s1]
    ])
    ez=np.array([
        [s4*s2, s3*s1],
        [s1*s3, s2*s4]
    ])
    mat=np.array([
        [ex[0,0]*ez[0,0], ex[0,0]*ez[0,1]],
        [ex[0,0]*ez[1,0], ex[0,0]*ez[1,1]]
    ])
    return mat

In [12]:
def eyey_matrix(s1,s2,s3,s4):
    ey=np.array([
        [s4*s1, s3*s2],
        [s1*s4, s2*s3]
    ])
    mat=np.array([
        [ey[0,0]*ey[0,0], ey[0,0]*ey[0,1]],
        [ey[0,0]*ey[1,0], ey[0,0]*ey[1,1]]
    ])
    return mat

In [13]:
def eyez_matrix(s1,s2,s3,s4):
    ey=np.array([
        [s4*s1, s3*s2],
        [s1*s4, s2*s3]
    ])
    ez=np.array([
        [s4*s2, s3*s1],
        [s1*s3, s2*s4]
    ])
    mat=np.array([
        [ey[0,0]*ez[0,0], ey[0,0]*ez[0,1]],
        [ey[0,0]*ez[1,0], ey[0,0]*ez[1,1]]
    ])
    return mat

In [14]:
def ezez_matrix(s1,s2,s3,s4):
    ez=np.array([
        [s4*s2, s3*s1],
        [s1*s3, s2*s4]
    ])
    mat=np.array([
        [ez[0,0]*ez[0,0], ez[0,0]*ez[0,1]],
        [ez[0,0]*ez[1,0], ez[0,0]*ez[1,1]]
    ])
    return mat

In [15]:
def ss_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=ss_matrix(0.0,0.0,0.0,0.0)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:

                    tmp = ss_matrix(s1,s2,s3,s4)
                    numer += tmp*np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
                    denom += np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
    
    res = numer/denom
    return res

In [16]:
def exex_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=exex_matrix(0.0,0.0,0.0,0.0)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:

                    tmp = exex_matrix(s1,s2,s3,s4)
                    numer += tmp*np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
                    denom += np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
    
    res = numer/denom
    return res

In [17]:
def exey_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=exey_matrix(0.0,0.0,0.0,0.0)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:

                    tmp = exey_matrix(s1,s2,s3,s4)
                    numer += tmp*np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
                    denom += np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
    
    res = numer/denom
    return res

In [18]:
def exez_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=exez_matrix(0.0,0.0,0.0,0.0)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:

                    tmp = exez_matrix(s1,s2,s3,s4)
                    numer += tmp*np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
                    denom += np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
    
    res = numer/denom
    return res

In [19]:
def eyey_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=eyey_matrix(0.0,0.0,0.0,0.0)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:

                    tmp = eyey_matrix(s1,s2,s3,s4)
                    numer += tmp*np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
                    denom += np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
    
    res = numer/denom
    return res

In [20]:
def eyez_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=eyez_matrix(0.0,0.0,0.0,0.0)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:

                    tmp = eyez_matrix(s1,s2,s3,s4)
                    numer += tmp*np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
                    denom += np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
    
    res = numer/denom
    return res

In [21]:
def ezez_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=ezez_matrix(0.0,0.0,0.0,0.0)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:

                    tmp = ezez_matrix(s1,s2,s3,s4)
                    numer += tmp*np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
                    denom += np.exp(-H2x2_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4))
    
    res = numer/denom
    return res

In [22]:
# print(Z2x2_numeric(beta,h,kx,ky,kz))
# print(Z2x2(beta,h,kx,ky,kz))

# print(mag_per_site2x2_numeric(beta,h,kx,ky,kz))
# print(mag_per_site2x2(beta,h,kx,ky,kz))

In [23]:
beta=0.474653
h=0.

kx=0.2
ky=2.1
kz=1.3

# kx=1.0
# ky=1.0
# kz=1.0

In [24]:
nin=1000
nfin=10000
nint=1
binsize=10

In [25]:
mag_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/mag/"+str(n)+".dat")
    mag_.append(dat)
    
mag=np.array(mag_)

In [26]:
jk_mag = Jackknife(mag.size, binsize)
jk_mag.set(simple_mean, mag)
jk_mag.do_it()

In [27]:
print("mag_per_site")
format_print_w_exact(0.0, jk_mag.mean(), jk_mag.err())

mag_per_site


'0.0024(35) exact:0.00000 [0.69 sigma]'

In [28]:
jk_magsq = Jackknife(mag.size, binsize)
jk_magsq.set(simple_mean, mag**2/4)
jk_magsq.do_it()

In [29]:
print("susc")
exact=susc2x2(beta,h,kx,ky,kz)
format_print_w_exact(exact, jk_magsq.mean(), jk_magsq.err())

susc


'3.9717(34) exact:3.97321 [0.46 sigma]'

In [30]:
exex_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/exex/"+str(n)+".dat")
    exex_.append(dat)
    
exex=np.array(exex_)

jk_exex = Jackknife(exex.size, binsize)
jk_exex.set(simple_mean, exex)
jk_exex.do_it()

print(format_print_w_exact_matrix(exex_corr_numeric(beta, h, kx,ky,kz), jk_exex.mean(), jk_exex.err()))

[['1.0 exact:0.9914630931528331' '1.0 exact:0.9914630931528331']
 ['0.9909(14) exact:1.00000 [6.39 sigma]'
  '0.9909(14) exact:1.00000 [6.39 sigma]']]


In [31]:
exey_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/exey/"+str(n)+".dat")
    exey_.append(dat)
    
exey=np.array(exey_)

jk_exey = Jackknife(exey.size, binsize)
jk_exey.set(simple_mean, exey)
jk_exey.do_it()

print(format_print_w_exact_matrix(exey_corr_numeric(beta, h, kx,ky,kz), jk_exey.mean(), jk_exey.err()))

[['0.9883(15) exact:0.98906 [0.50 sigma]'
  '0.9883(15) exact:0.98906 [0.50 sigma]']
 ['0.9883(15) exact:0.98906 [0.50 sigma]'
  '0.9883(15) exact:0.98906 [0.50 sigma]']]


In [32]:
exez_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/exez/"+str(n)+".dat")
    exez_.append(dat)
    
exez=np.array(exez_)

jk_exez = Jackknife(exez.size, binsize)
jk_exez.set(simple_mean, exez)
jk_exez.do_it()

print(format_print_w_exact_matrix(exez_corr_numeric(beta, h, kx,ky,kz), jk_exez.mean(), jk_exez.err()))

[['0.99522(75) exact:0.995407 [0.25 sigma]'
  '0.99522(75) exact:0.995407 [0.25 sigma]']
 ['0.99522(75) exact:0.995407 [0.25 sigma]'
  '0.99522(75) exact:0.995407 [0.25 sigma]']]


In [33]:
eyey_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/eyey/"+str(n)+".dat")
    eyey_.append(dat)
    
eyey=np.array(eyey_)

jk_eyey = Jackknife(eyey.size, binsize)
jk_eyey.set(simple_mean, eyey)
jk_eyey.do_it()

print(format_print_w_exact_matrix(eyey_corr_numeric(beta, h, kx,ky,kz), jk_eyey.mean(), jk_eyey.err()))

[['1.0 exact:1.0' '0.9909(14) exact:0.99146 [0.40 sigma]']
 ['1.0 exact:1.0' '0.9909(14) exact:0.99146 [0.40 sigma]']]


In [34]:
eyez_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/eyez/"+str(n)+".dat")
    eyez_.append(dat)
    
eyez=np.array(eyez_)

jk_eyez = Jackknife(eyez.size, binsize)
jk_eyez.set(simple_mean, eyez)
jk_eyez.do_it()

print(format_print_w_exact_matrix(eyez_corr_numeric(beta, h, kx,ky,kz), jk_eyez.mean(), jk_eyez.err()))

[['0.9881(15) exact:0.98874 [0.43 sigma]'
  '0.9881(15) exact:0.98874 [0.43 sigma]']
 ['0.9881(15) exact:0.98874 [0.43 sigma]'
  '0.9881(15) exact:0.98874 [0.43 sigma]']]


In [35]:
ezez_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/ezez/"+str(n)+".dat")
    ezez_.append(dat)
    
ezez=np.array(ezez_)

jk_ezez = Jackknife(ezez.size, binsize)
jk_ezez.set(simple_mean, ezez)
jk_ezez.do_it()

print(format_print_w_exact_matrix(ezez_corr_numeric(beta, h, kx,ky,kz), jk_ezez.mean(), jk_ezez.err()))

[['1.0 exact:1.0' '0.9909(14) exact:0.99146 [0.40 sigma]']
 ['0.9909(14) exact:0.99146 [0.40 sigma]' '1.0 exact:1.0']]
